In [1]:
!pip install seqeval
!pip install vncorenlp
!pip install underthesea
!git clone https://github.com/vncorenlp/VnCoreNLP.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=732aa23b46f825661f9b74b9d4a720a692c7416e608684614c70f48341ca968b
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645933 sha256=c752be532e39d8c7d27167fae835f7941fd9c43c28f23beeb4dc3857fab542a2
  Stored in directory: /root/.cache/pip/wheels/80/ad/d4/9e1a0939f63331a3898f2a951a368bbf0d69f7b027cae4d66b
Successfully built vncorenlp
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [2]:
!wget https://github.com/vncorenlp/VnCoreNLP/archive/refs/tags/v1.2.zip
!unzip -o v1.2.zip 


--2025-07-07 21:32:10--  https://github.com/vncorenlp/VnCoreNLP/archive/refs/tags/v1.2.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/vncorenlp/VnCoreNLP/zip/refs/tags/v1.2 [following]
--2025-07-07 21:32:10--  https://codeload.github.com/vncorenlp/VnCoreNLP/zip/refs/tags/v1.2
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v1.2.zip’

v1.2.zip                [         <=>        ] 160.83M  25.9MB/s    in 6.2s    

2025-07-07 21:32:17 (25.9 MB/s) - ‘v1.2.zip’ saved [168648673]

Archive:  v1.2.zip
62bbc58fe5d113c898eae112656be97dcf50b3a0
   creating: VnCoreNLP-1.2/
  inflating: VnCoreNLP-1.2/LICENSE.md  
  inflating: VnCoreNL

In [3]:
import random, os
import numpy as np
import torch
import pandas as pd
from ast import literal_eval
from io import StringIO
from underthesea import ner, pos_tag
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score

from vncorenlp import VnCoreNLP
from tqdm.notebook import tqdm
import time

In [4]:
VNCORENLP_DIR = os.path.join("/kaggle/working/VnCoreNLP-1.2/VnCoreNLP-1.2.jar")

In [5]:
annotator = VnCoreNLP(VNCORENLP_DIR)

# Load data VLSP_2016_NER

In [6]:

df_train_dir = '../input/vlsp-2016-ner/train.csv'
df_test_dir = '../input/vlsp-2016-ner/test.csv'
df_dev_dir = '../input/vlsp-2016-ner/dev.csv'


In [7]:

def load_df(file_dir):
    # Đọc toàn bộ file vào chuỗi
    with open(file_dir, encoding='utf-8') as f:
        text = f.read()
    
    # Tách các đoạn theo '\n\n'
    samples = text.strip().split('\n\n')
    
    # Chuyển mỗi đoạn thành DataFrame, lưu vào list
    dataframes = []
    for sample in samples:
        # Bỏ qua đoạn rỗng
        if not sample.strip():
            continue
        # Đọc đoạn thành DataFrame tạm thời
        df = pd.read_csv(StringIO(sample), sep='\t', skip_blank_lines=True,
                         header=None, names=['word', 'pos', 'chunk', 'ner']) 
        df = df.dropna().reset_index(drop=True)

        dataframes.append(df)
    return dataframes
df_test = load_df(df_test_dir)
df_dev = load_df(df_dev_dir)

print(df_test[0])

        word  pos chunk    ner
0   Trà_Vinh  NNP  B-NP  B-LOC
1        thì    C     O      O
2         cử    V  B-VP      O
3        sáu    M  B-NP      O
4   đồng_chí    N  B-NP      O
5          ,   CH     O      O
6       mang    V  B-VP      O
7    bí_danh    N  B-NP      O
8         là    C     O      O
9       Đoàn  NNP  B-NP  B-PER
10         ,   CH     O      O
11       Kết  NNP  B-NP  B-PER
12         ,   CH     O      O
13       Đấu  NNP  B-NP  B-PER
14         ,   CH     O      O
15     Tranh  NNP  B-NP  B-PER
16         ,   CH     O      O
17     Thắng  NNP  B-NP  B-PER
18         ,   CH     O      O
19       Lợi  NNP  B-NP  B-PER
20         .   CH     O      O


In [8]:
len(df_test)

2530

In [9]:
for i, df in enumerate(df_test):
    if df['word'].isna().any() or df['ner'].isna().any():
        print(f"\nSample {i} chứa NaN toàn bộ câu:")
        print(df)

In [10]:
def read_conll_data(df_list):
    sentences,pos_labels, ner_labels = [],  [],  []
    for df in df_list:
        sentences.append(df["word"].tolist())
        ner_labels.append(df["ner"].tolist())
        pos_labels.append(df["pos"].tolist())
        
    return sentences, pos_labels, ner_labels

In [11]:
sentences, true_pos_labels, true_ner_labels = read_conll_data(df_test)
true_pos_labels = [['Np' if tag == 'NNP' else tag for tag in pos_tags]for pos_tags in true_pos_labels]

In [12]:
print("Số câu:", len(sentences))
print("Câu đầu tiên:", sentences[0])
print("POS đầu tiên:", true_pos_labels[0])
print("NER đầu tiên:", true_ner_labels[0])

Số câu: 2530
Câu đầu tiên: ['Trà_Vinh', 'thì', 'cử', 'sáu', 'đồng_chí', ',', 'mang', 'bí_danh', 'là', 'Đoàn', ',', 'Kết', ',', 'Đấu', ',', 'Tranh', ',', 'Thắng', ',', 'Lợi', '.']
POS đầu tiên: ['Np', 'C', 'V', 'M', 'N', 'CH', 'V', 'N', 'C', 'Np', 'CH', 'Np', 'CH', 'Np', 'CH', 'Np', 'CH', 'Np', 'CH', 'Np', 'CH']
NER đầu tiên: ['B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'B-PER', 'O', 'B-PER', 'O', 'B-PER', 'O', 'B-PER', 'O', 'B-PER', 'O']


In [13]:
from sklearn.metrics import classification_report as sk_classification_report
from sklearn.metrics import  precision_score as sk_precision_score, f1_score as sk_f1_score, recall_score as sk_recall_score
def eval_seq(true_labels, pred_labels, task_name="ner"):
    print(f"\nĐÁNH GIÁ CHO TASK: {task_name.upper()}")
    print(f"Precision: {precision_score(true_labels, pred_labels) * 100:.2f}")
    print(f"Recall:    {recall_score(true_labels, pred_labels) * 100:.2f}")
    print(f"F1-score:  {f1_score(true_labels, pred_labels) * 100:.2f}")
    print("\nBáo cáo chi tiết (seqeval):")
    print(classification_report(true_labels, pred_labels)) 
    
def eval_sk(true_labels, pred_labels, task_name="ner"):
    true_flat = [tag for sent in true_labels for tag in sent]
    pred_flat = [tag for sent in pred_labels for tag in sent]
    print(f"\nĐÁNH GIÁ CHO TASK: {task_name.upper()}")
    print(f"Precision: {sk_precision_score(true_flat, pred_flat, average='macro') * 100:.2f}")
    print(f"Recall:    {sk_recall_score(true_flat, pred_flat, average='macro') * 100:.2f}")
    print(f"F1-score:  {sk_f1_score(true_flat, pred_flat, average='macro') * 100:.2f}")
    print("\nBáo cáo chi tiết (với sklearn):")    
    print(sk_classification_report(true_flat, pred_flat))

def export_errors_to_csv(sentences, true_labels, pred_labels, output_file="errors.csv"):
   
    errors = []
    for i, (words, gold, pred) in enumerate(zip(sentences, true_labels, pred_labels)):
        for j, (w, g, p) in enumerate(zip(words, gold, pred)):
            if g != p:
                errors.append({
                    "sentence_id": i,
                    "position": j,
                    "word": w,
                    "true_label": g,
                    "predicted_label": p,
                })

    df_errors = pd.DataFrame(errors)
    df_errors.to_csv(output_file, index=False)
    print(f"Xuất {len(df_errors)} lỗi sai ra file: {output_file}")
    return df_errors

# NER

In [14]:

def predict_ner(sentences, tool = "underthesea"):
    predicted_labels = []
    if tool != "underthesea" and tool != "VnCoreNLP":
        print('Tool không hợp lệ')
        return
    
    for sent in tqdm(sentences): 
        labels = []     
        tagged_dict ={}
        if tool == "underthesea":
            tagged = ner(' '.join(sent).replace("_", " "))
            tagged_dict = {item[0].replace(' ', '_'): item[3] for item in tagged}
        elif tool == "VnCoreNLP":
            tagged = annotator.ner(' '.join(sent).replace("_", " "))
            tagged_dict = {item[0]: item[1] for sen in tagged for item in sen}
        for token in sent:
            labels.append(tagged_dict.get(token, "O"))        
        predicted_labels.append(labels)
    
    return predicted_labels


## underthesea

In [15]:
pred_labels = predict_ner(sentences)

  0%|          | 0/2530 [00:00<?, ?it/s]

In [16]:
eval_seq(true_ner_labels, pred_labels, "NER với underthesea")
eval_sk(true_ner_labels, pred_labels, "NER với underthesea")


ĐÁNH GIÁ CHO TASK: NER VỚI UNDERTHESEA
Precision: 54.09
Recall:    53.36
F1-score:  53.72

Báo cáo chi tiết (seqeval):
              precision    recall  f1-score   support

         LOC       0.39      0.64      0.49       887
        MISC       0.29      0.21      0.24        39
         ORG       0.59      0.10      0.17       200
         PER       0.89      0.53      0.67      1091

   micro avg       0.54      0.53      0.54      2217
   macro avg       0.54      0.37      0.39      2217
weighted avg       0.65      0.53      0.54      2217


ĐÁNH GIÁ CHO TASK: NER VỚI UNDERTHESEA
Precision: 62.42
Recall:    38.67
F1-score:  38.31

Báo cáo chi tiết (với sklearn):
              precision    recall  f1-score   support

       B-LOC       0.44      0.65      0.52       887
      B-MISC       0.41      0.23      0.30        39
       B-ORG       0.79      0.14      0.23       200
       B-PER       0.90      0.54      0.68      1091
       I-LOC       0.19      0.60      0.29       

In [17]:
export_errors_to_csv(sentences, true_ner_labels, pred_labels, output_file="errors_NER_underthesea.csv")

Xuất 3110 lỗi sai ra file: errors_NER_underthesea.csv


,sentence_id,position,word,true_label,predicted_label
0,0,11,Kết,B-PER,B-LOC
1,0,13,Đấu,B-PER,I-LOC
2,0,15,Tranh,B-PER,I-LOC
3,0,19,Lợi,B-PER,I-LOC
4,6,18,núi,O,B-LOC
...,...,...,...,...,...
3105,2517,12,Diệc,I-PER,O
3106,2517,13,Đình,I-PER,O
3107,2519,8,gia_đình,O,B-LOC
3108,2519,9,Đức,B-PER,I-LOC


## VnCoreNLP

In [18]:

pred_labels = predict_ner(sentences, tool = "VnCoreNLP")


  0%|          | 0/2530 [00:00<?, ?it/s]

In [19]:
eval_seq(true_ner_labels, pred_labels, "NER với VNCoreNLP")
eval_sk(true_ner_labels, pred_labels, "NER với VNCoreNLP")



ĐÁNH GIÁ CHO TASK: NER VỚI VNCORENLP
Precision: 90.24
Recall:    73.43
F1-score:  80.97

Báo cáo chi tiết (seqeval):
              precision    recall  f1-score   support

         LOC       0.90      0.84      0.87       887
        MISC       0.61      0.79      0.69        39
         ORG       0.72      0.65      0.68       200
         PER       0.96      0.66      0.79      1091

   micro avg       0.90      0.73      0.81      2217
   macro avg       0.80      0.74      0.76      2217
weighted avg       0.91      0.73      0.81      2217


ĐÁNH GIÁ CHO TASK: NER VỚI VNCORENLP
Precision: 91.03
Recall:    71.50
F1-score:  75.52

Báo cáo chi tiết (với sklearn):
              precision    recall  f1-score   support

       B-LOC       0.95      0.87      0.91       887
      B-MISC       0.74      0.82      0.78        39
       B-ORG       0.94      0.75      0.84       200
       B-PER       0.98      0.67      0.80      1091
       I-LOC       0.96      0.62      0.75       367


In [20]:
export_errors_to_csv(sentences, true_ner_labels, pred_labels, output_file="errors_NER_VNCoreNLP.csv")

Xuất 1271 lỗi sai ra file: errors_NER_VNCoreNLP.csv


,sentence_id,position,word,true_label,predicted_label
0,0,15,Tranh,B-PER,O
1,9,30,Việt,B-LOC,O
2,23,23,Bạch,B-PER,O
3,23,24,Tuyết,I-PER,O
4,38,1,Bùi,B-PER,O
...,...,...,...,...,...
1266,2517,11,Lưu,B-PER,O
1267,2517,12,Diệc,I-PER,O
1268,2517,13,Đình,I-PER,O
1269,2519,4,xã,B-LOC,O


# POS tagging

In [21]:
def predict_pos(sentences, tool = "underthesea"):
    predicted_labels = []
    if tool != "underthesea" and tool != "VnCoreNLP":
        print('Tool không hợp lệ')
        return None
    sentences = [[word.replace("_", " ") for word in sent] for sent in sentences]    
    for sent in tqdm(sentences):
        labels = []     
        tagged_dict ={}
        if tool == "underthesea":
            tagged = pos_tag(' '.join(sent))
            tagged_dict = {item[0]: item[1] for item in tagged}
        elif tool == "VnCoreNLP":
            tagged = annotator.pos_tag(' '.join(sent))
            tagged_dict = {item[0].replace('_', ' '): item[1] for sen in tagged for item in sen}
        for token in sent:
            labels.append(tagged_dict.get(token, "O"))        
        predicted_labels.append(labels)
    
    return predicted_labels

## POS tagging - underthesea

In [22]:
pred_labels = predict_pos(sentences)

  0%|          | 0/2530 [00:00<?, ?it/s]

In [23]:
eval_sk(true_pos_labels, pred_labels, "POS tagging với underthesea")


ĐÁNH GIÁ CHO TASK: POS TAGGING VỚI UNDERTHESEA


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 53.68


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recall:    51.69
F1-score:  51.61

Báo cáo chi tiết (với sklearn):


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           A       0.94      0.79      0.86      3260
          Ab       0.00      0.00      0.00         2
           C       0.65      0.90      0.76      1523
          CH       1.00      1.00      1.00      7825
          Cc       0.00      0.00      0.00       644
           E       0.94      0.96      0.95      3474
          FW       0.05      0.09      0.07        35
           I       0.87      0.72      0.79        36
        I-NP       0.00      0.00      0.00         1
           L       0.99      0.97      0.98      1049
           M       0.98      0.94      0.96      1940
           N       0.95      0.88      0.92     13088
         NPP       0.00      0.00      0.00         2
          Nb       0.00      0.00      0.00        74
          Nc       0.21      0.77      0.33       477
          Ne       0.00      0.00      0.00        81
          Ni       0.00      0.00      0.00         5
          Np       0.95    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.

In [24]:
export_errors_to_csv(sentences, true_pos_labels, pred_labels, output_file="errors_POS_underthesea.csv")

Xuất 7979 lỗi sai ra file: errors_POS_underthesea.csv


,sentence_id,position,word,true_label,predicted_label
0,0,8,là,C,V
1,1,1,tỉnh_uỷ,N,O
2,2,11,anh,Ns,Nc
3,2,22,anh,Ns,Nc
4,3,17,thêm,A,R
...,...,...,...,...,...
7974,2526,5,và,Cc,C
7975,2526,12,tuyệt_đối,V,A
7976,2527,10,người_ta,P,O
7977,2529,3,và,Cc,C


## POS tagging - VnCoreNLP

In [25]:
pred_labels = predict_pos(sentences, tool = "VnCoreNLP")

  0%|          | 0/2530 [00:00<?, ?it/s]

In [26]:
eval_sk(true_pos_labels, pred_labels, "POS tagging với VnCoreNLP")


ĐÁNH GIÁ CHO TASK: POS TAGGING VỚI VNCORENLP


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 60.50


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recall:    60.60
F1-score:  59.34

Báo cáo chi tiết (với sklearn):


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           A       0.93      0.92      0.93      3260
          Ab       1.00      0.50      0.67         2
           C       0.92      0.92      0.92      1523
          CH       1.00      0.98      0.99      7825
          Cc       1.00      0.93      0.96       644
           E       0.91      0.95      0.93      3474
          Eb       0.00      0.00      0.00         0
          FW       0.00      0.00      0.00        35
           I       0.87      0.72      0.79        36
        I-NP       0.00      0.00      0.00         1
           L       0.99      0.95      0.97      1049
           M       0.94      0.98      0.96      1940
           N       0.92      0.91      0.92     13088
         NPP       0.00      0.00      0.00         2
          Nb       0.87      0.72      0.79        74
          Nc       0.32      0.90      0.47       477
          Ne       0.00      0.00      0.00        81
          Ni       0.00    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.

In [27]:
export_errors_to_csv(sentences, true_pos_labels, pred_labels, output_file="errors_POS_VnCoreNLP.csv")

Xuất 6058 lỗi sai ra file: errors_POS_VnCoreNLP.csv


,sentence_id,position,word,true_label,predicted_label
0,2,11,anh,Ns,N
1,2,22,anh,Ns,N
2,3,17,thêm,A,R
3,4,0,Về,V,E
4,4,3,ông,Ns,Nc
...,...,...,...,...,...
6053,2526,12,tuyệt_đối,V,A
6054,2527,10,người_ta,P,O
6055,2528,11,VN,Np,Ny
6056,2529,1,người,N,O
